- [previous file - EDA](2022-03-31_train-test_EDA.ipynb)
- [next tile - models with pre-tuned parameters and ensembles](2022-04-15_ensemble.ipynb)

## imports

In [4]:
from warnings import filterwarnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

from lib.model_related import *

sns.set()
filterwarnings("ignore")

## reading data

In [5]:
train_raw = pd.read_parquet("data/2022-04-08_train_pre-model.parquet")
test_raw = pd.read_parquet("data/2022-04-08_test_pre-model.parquet")

train_raw.shape, test_raw.shape

((115367, 30), (34686, 28))

In [6]:
def submit(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = preds
    submission.to_csv(f"{name}.csv", index=False)
    
    
def submit_log(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = np.exp(preds)
    submission.to_csv(f"{name}.csv", index=False)

In [7]:
train_raw["price"].sum()

158409758714.0

## encoding

In [7]:
train_raw["train/test"] = "train"
test_raw["train/test"] = "test"

data = train_raw.append(test_raw)
data["ptc"].fillna("Оригинал", inplace=True)

data[data.select_dtypes("object").columns.tolist()] = data[
    data.select_dtypes("object").columns.tolist()
].astype(str)

for col in set(data.select_dtypes(exclude=("object")).columns) - {"price"}:
    data[col] = (
        RobustScaler().fit_transform(data[col].values.reshape(-1, 1)).reshape(-1, 1)
    )

for col in ["model_name"]:
    data[col] = LabelEncoder().fit_transform(data[col].astype("str"))

data = pd.get_dummies(
    data,
    columns=[
        "vehicle_transmission",
        "vendor",
        "brand",
        "fuel_type",
        "body_type",
        "color",
        "ptc",
        "drive",
        "wheel",
        "age_cat",
    ],
)

train = data.loc[data["train/test"] == "train"]

train_jane = train.loc[train["sample"] == "jane"]
train_sokolov = train.loc[train["sample"] == "sokolov"]
train_jane["price"] = train_jane["price"] * 0.86
train = train_jane.append(train_sokolov)

train.drop(columns=["sample", "description", "train/test"], inplace=True)
test = data.loc[data["train/test"] == "test"].drop(
    columns=["sample", "description", "train/test", "price"]
)

## modelling

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns="price"), train["price"], random_state = 42, shuffle=True)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((86525, 112), (86525,), (28842, 112), (28842,))

## model tuning

### KNN

In [9]:
def objective(trial):

    param = {
        'n_neighbors': trial.suggest_int("n_neighbors", 1, 30),
        'leaf_size': trial.suggest_int("leaf_size", 1, 50),
        'p': trial.suggest_int("p", 1, 2),
        'weights': trial.suggest_categorical("weights", ['uniform', 'distance']),
        'metric': trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    }

    knn = KNeighborsRegressor(**param)
    cv_roc_auc = cross_val_score(knn, X_train, y_train, cv=3, scoring="neg_mean_absolute_percentage_error", n_jobs=-1)

    return np.mean(cv_roc_auc)


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///KNNRegressor.db",
    study_name="KNNRegressor",
    load_if_exists=True,
)
study.optimize(objective, n_trials=2)

[I 2022-04-13 18:13:06,186] Using an existing study with name 'KNNRegressor' instead of creating a new one.
[I 2022-04-13 18:23:22,123] Trial 47 finished with value: -0.1642623637534808 and parameters: {'n_neighbors': 7, 'leaf_size': 38, 'p': 2, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 44 with value: -0.15464020043822252.
[I 2022-04-13 18:33:32,685] Trial 48 finished with value: -0.1582071567694789 and parameters: {'n_neighbors': 8, 'leaf_size': 45, 'p': 2, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 44 with value: -0.15464020043822252.


In [10]:
study.best_params

{'leaf_size': 41,
 'metric': 'manhattan',
 'n_neighbors': 8,
 'p': 2,
 'weights': 'distance'}

#### Version 1

In [11]:
knn_optuned_32 = KNeighborsRegressor(
    **{
        'n_neighbors': 9, 
        'leaf_size': 41, 
        'p': 1, 
        'weights': 'distance', 
        'metric': 'manhattan'
    }
).fit(X_train, np.log(y_train))

print("knn_optuned_32", mean_absolute_percentage_error(y_valid, np.exp(knn_optuned_32.predict(X_valid))))
submit_log(test, knn_optuned_32, "knn_optuned_32")

knn_optuned_32 0.14258679506496774
